# **사업 분야 확장을 위한 고객 기반 카테고리 분석**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')


MessageError: Error: credential propagation was unsuccessful

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#**데이터 전처리**

##카테고리 이름 영어로 변경
(제품 카테고리명이 포르투갈어로 되어 있음, 영어로 번역된 데이터셋으로 대체)

In [ ]:
products = products.merge(category, left_on='product_category_name',right_on='product_category_name', how='left')

In [ ]:
products = products.drop(['product_category_name'], axis=1)

##병합
1.주문 테이블 + 주문 상품 테이블

In [ ]:
order_products_merged=order_items.merge(products, on='product_id',how='left')

In [ ]:
order_products_merged

2.주문 테이블 + 고객 테이블

In [ ]:
customers=customers.merge(orders,on='customer_id',how='left')

In [ ]:
customers

3. 1과2를 병합

(고객별 주문 내역과 주문 상품이 포함된 테이블 생성)



In [ ]:
customers_orders=customers.merge(order_products_merged,on='order_id',how='left')

In [ ]:
customers_orders

##필요한 컬럼만 남기기
(가설 검증에 불필요한 상품의 무게,배송 날짜 등의 컬럼 삭제)

In [ ]:
# 필요한 컬럼 선택
selected_columns = [
    'order_id',
    'customer_unique_id',
    'customer_zip_code_prefix',
    'customer_city',
    'customer_state',
    'order_item_id',
    'product_id',
    'product_category_name_english'
]

# 새로운 데이터프레임 생성
customers_orders_new= customers_orders[selected_columns]


In [ ]:
customers_orders_new

In [ ]:
# 결측치의 총 갯수 확인하기
customers_orders_new.isna().sum()

결측치 날리기
* 임의로 채우면 오히려 노이즈가 발생할 확률이 높다고 판단, 제거해도 치명적인 데이터 양이 아니라 drop 결정





# **STEP01**

state의 수가 27개로 너무 많아서,  

*   북부 (North)	AC,AM, AP, PA, RO, RR, TO
*   북동부 (Northeast)	AL, BA, CE, MA, PB, PE, PI, RN, SE
*   중서부 (Central-West)	DF, GO, MT, MS
*   남동부 (Southeast)	ES, MG, RJ, SP
*   남부 (South)	PR, RS, SC

지역별로 추려서 5개의 그룹으로 그룹할 예정






In [ ]:
# 주(State)별 지역 그룹화
region_mapping = {
    "North": ["AC", "AM", "AP", "PA", "RO", "RR", "TO"],
    "Northeast": ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
    "Central-West": ["DF", "GO", "MT", "MS"],
    "Southeast": ["ES", "MG", "RJ", "SP"],
    "South": ["PR", "RS", "SC"]
}

# 데이터프레임에 지역 추가
customers_orders_new["region"] = customers_orders_new["customer_state"].map(lambda x: next((k for k, v in region_mapping.items() if x in v), "Unknown"))

# 1. 지역(Region)별 인기 제품 분석
region_popular_products = customers_orders_new.groupby(['region', 'product_category_name_english']).size().reset_index(name='purchase_count')

# 각 지역에서 가장 많이 판매된 제품 카테고리 찾기
top_products_per_region = region_popular_products.sort_values(['region', 'purchase_count'], ascending=[True, False]).groupby('region').head(5).reset_index()

# 2. 특정 지역 고객의 구매 패턴 분석
# 고객이 구매한 제품과 지역별 인기 제품 매칭
customer_purchases = customers_orders_new.merge(top_products_per_region, on='region', suffixes=('_customer', '_region'))




In [ ]:
top_products_per_region
#각 지역에서 가장 많이 판매된 카테고리

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# "Southeast"가 포함된 모든 지역 데이터 필터링
southeast_data = top_products_per_region[top_products_per_region['region'].str.contains("Southeast")]

# 그래프 크기 설정
fig, ax = plt.subplots(figsize=(14, 6))  # 단일 플롯이므로 axes가 아닌 ax 사용

# Southeast 지역 그래프
sns.barplot(
    data=southeast_data,
    x='region',
    y='purchase_count',
    hue='product_category_name_english',
    dodge=True,
    ax=ax  # 단일 플롯이므로 ax 사용
)

# 그래프 제목 및 라벨
ax.set_title("Top 5 Most Sold Product Categories in Southeast", fontsize=14)
ax.set_xlabel("Region", fontsize=12)
ax.tick_params(axis='x', rotation=45)
ax.legend(title="Product Category")

# 레이아웃 조정 및 출력
plt.tight_layout()
plt.show()



# **STEP02**

## 동일한 테이블 사용하므로 병합 과정 생략

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Brazilian E-Commerce Public Dataset/eng_ver.csv')

#원본 백업
df_origin = df.copy()

In [ ]:
#전체 컬럼 보기 설정
pd.set_option('display.max_columns', None)
#필수 컬럼만 남김
df = df[['order_id', 'customer_unique_id', 'product_id', 'order_item_id', 'product_category_name_english']]
#컬럼명 변경
df.rename(columns={'product_category_name_english': 'product_category_name'}, inplace=True)


df.info()

## 결측치

In [ ]:
df.isna().sum()

In [ ]:
df.isna().mean()

In [ ]:
df[df['product_category_name'].isna()]

임의로 채우면 오히려 노이즈가 발생할 확률이 높다고 판단, 제거해도 치명적인 데이터 양이 아니라 drop 결정

In [ ]:
product_category_name_nan = df[df['product_category_name'].isna()].index
product_id_nan = df[df['product_id'].isna()].index
order_item_id_nan = df[df['order_item_id'].isna()].index

In [ ]:
len(df)

In [ ]:
df.dropna()

## 재구매율 구하기
→ 전체 고객 중, 특정 카테고리를 한 번 이상 반복 구매한 고객의 비율

In [ ]:
#고객별로 구매한 카테고리 개수 계산
customer_category_counts = df.groupby(['customer_unique_id', 'product_category_name']).size().reset_index(name='purchase_count')
customer_category_counts

In [ ]:
#구매 횟수가 2회 이상인 경우
repeat_purchases = customer_category_counts[customer_category_counts['purchase_count'] > 1]
repeat_purchases

In [ ]:
#재구매 고객 수
repeat_customer_count = repeat_purchases['customer_unique_id'].nunique()
repeat_customer_count

In [ ]:
#전체 고객 수
total_customers = df['customer_unique_id'].nunique()
total_customers

In [ ]:
#재구매율
repeat_purchase_rate = repeat_customer_count / total_customers
print(repeat_purchase_rate)
round(repeat_purchase_rate*100,2)

## 카테고리별 재구매율 확인
### 목적
어떤 제품군이 재구매가 활발한지 분석하여 **전략 수립** 가능
### 방법
각 product_category_name별로 고객이 동일한 카테고리를 다시 구매한 비율을 계산

In [ ]:
# 각 카테고리별 고객의 구매 횟수 계산
category_repeat_counts = df.groupby(['product_category_name', 'customer_unique_id']).size().reset_index(name='purchase_count')

# 각 카테고리에서 재구매한 고객 수 계산
category_repurchase = category_repeat_counts[category_repeat_counts['purchase_count'] > 1]
category_repurchase_rate = (
    category_repurchase.groupby('product_category_name')['customer_unique_id'].nunique()
    / category_repeat_counts.groupby('product_category_name')['customer_unique_id'].nunique()
) * 100

# 특정 카테고리만 필터링(bed_bath_table을 제외한 Step01의 TOP5 카테고리)
selected_categories = ['furniture_decor', 'health_beauty', 'sports_leisure', 'computers_accessories']
filtered_repurchase_rate = category_repurchase_rate[category_repurchase_rate.index.isin(selected_categories)]
filtered_repurchase_rate

In [ ]:
# Seaborn을 활용한 시각화
import seaborn as sns
import matplotlib.pyplot as plt

# Seaborn 스타일 설정
sns.set_style("whitegrid")

# 그래프 그리기
plt.figure(figsize=(8, 5))
ax = sns.barplot(x=filtered_repurchase_rate.values, y=filtered_repurchase_rate.index, palette="viridis")

# 레이블 추가
plt.xlabel("Repurchase Rate (%)")
plt.ylabel("Product Category")
plt.title("Repurchase Rate by Selected Categories")

# 수치 정확하게 표시
for index, value in enumerate(filtered_repurchase_rate):
    ax.text(value + 0.5, index, f"{value:.2f}%", va='center', fontsize=10)

plt.xlim(0, max(filtered_repurchase_rate) + 5)
plt.show()


→ 가장 재구매율이 높은 것은 **furniture_decor**로 확인.

## 통계적 검증
고객이 동일 카테고리를 반복 구매할 확률이 무작위보다 높은지 검정하고자 함.

### 카이제곱검정
→ 고객의 재구매 여부는 범주형 데이터이므로 두 변수가 서로 독립적인지 확인
- **변수 1:** 고객이 특정 카테고리를 재구매했는가? (Yes / No)
- **변수 2:** 제품 카테고리


- **귀무가설 (H₀):** 고객이 과거에 구매한 카테고리를 다시 구매하는 것은 **랜덤**이다.
    
    → 고객이 이전에 구매한 카테고리와 관계없이 아무 카테고리나 다시 구매할 가능성이 동일하다.
    
- **대립가설 (H₁):** 고객은 과거에 구매한 카테고리를 다시 구매할 가능성이 **유의미하다.**
    
    → 특정 카테고리를 한 번 구매한 고객은 다시 같은 카테고리를 구매하는 경향이 있다.
        
따라서 p-value가 0.05보다 작다면 "고객이 과거에 구매한 카테고리를 다시 구매하는 경향이 있다"고 판단할 수 있다.

In [ ]:
from scipy.stats import chi2_contingency

# 재구매 여부를 나타내는 컬럼 추가
df['repeat_purchase'] = df.duplicated(subset=['customer_unique_id', 'product_category_name'], keep=False)

# 교차표 생성
contingency_table = pd.crosstab(df['repeat_purchase'], df['product_category_name'])

# 카이제곱 검정 실행
chi2, p, dof, expected = chi2_contingency(contingency_table)

# 결과 출력
print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")

# 유의 수준 0.05 기준 검정 결과 해석
if p < 0.05:
    print("과거에 구매한 적 있는 카테고리에서 재구매할 가능성이 유의미하게 높음.")
else:
    print("우연적 결과.")


# **STEP03**

전환율 구하기

In [ ]:
df_new=pd.read_csv('/content/gdrive/MyDrive/MainQuest04/step02.csv')

In [ ]:
customer_category_counts = df_new.groupby(['customer_unique_id', 'product_category_name']).size().reset_index(name='purchase_count')

#여러 개의 카테고리를 구매한 고객만 필터링
multi_category_customers = customer_category_counts[customer_category_counts.duplicated(subset=['customer_unique_id'], keep=False)]

#각 고객이 구매한 모든 카테고리 조합을 생성
customer_category_pairs = multi_category_customers.groupby('customer_unique_id')['product_category_name'].apply(set).reset_index()

#모든 조합을 추출하여 빈도수 계산
from itertools import combinations
from collections import Counter

category_combinations = Counter()

for categories in customer_category_pairs['product_category_name']:
    for combo in combinations(categories, 2):
        category_combinations[frozenset(combo)] += 1

#데이터프레임으로 변환
category_combinations_df = pd.DataFrame(category_combinations.items(), columns=['Category Pair', 'Count'])
category_combinations_df = category_combinations_df.sort_values(by='Count', ascending=False)

#'bed_bath_table'과 함께 구매된 카테고리 필터링
bed_bath_related_combinations = category_combinations_df[category_combinations_df['Category Pair'].apply(lambda x: 'bed_bath_table' in x)].copy()

#'bed_bath_table'과 함께 구매된 카테고리를 추출
bed_bath_related_combinations['Other Category'] = bed_bath_related_combinations['Category Pair'].apply(lambda x: list(x - {'bed_bath_table'})[0])

#'bed_bath_table'의 총 구매 횟수 계산
bed_bath_total_sales = df_new[df_new['product_category_name'] == 'bed_bath_table'].shape[0]

#전환율(%) 계산
bed_bath_related_combinations['Conversion Rate (%)'] = (bed_bath_related_combinations['Count'] / bed_bath_total_sales) * 100

#가장 높은 전환율을 가진 카테고리 찾기
top_conversion_category = bed_bath_related_combinations.sort_values(by='Count', ascending=False).iloc[0]

#최종 결과 출력
top_conversion_category
bed_bath_related_combinations.head() # 여기서 말하는 전환률은 함께 구입한 경우의 수를 의미한다.

In [ ]:

# 1. 구매 이력 정리 (고객별 구매 순서 정렬)
if 'order_purchase_timestamp' in df_new.columns:
    df_new['order_purchase_timestamp'] = pd.to_datetime(df_new['order_purchase_timestamp'])
    df_sorted = df_new.sort_values(by=['customer_unique_id', 'order_purchase_timestamp'])
else:
    # 구매 날짜 정보가 없는 경우, order_id 기준 정렬
    df_sorted = df_new.sort_values(by=['customer_unique_id', 'order_id'])

# 2. 고객별 첫 번째 구매 카테고리 확인
df_sorted['previous_category'] = df_sorted.groupby('customer_unique_id')['product_category_name'].shift(1)

# 3. 'bed_bath_table' 구매 이후 다른 카테고리를 구매한 경우 필터링
df_transitions = df_sorted[(df_sorted['previous_category'] == 'bed_bath_table') &
                           (df_sorted['product_category_name'] != 'bed_bath_table')]

# 4. bed_bath_table 이후 가장 많이 구매된 카테고리 및 전환율 계산
transition_counts = df_transitions['product_category_name'].value_counts()
total_bed_bath_customers = df_sorted[df_sorted['product_category_name'] == 'bed_bath_table']['customer_unique_id'].nunique()

# 5. 전환율(%) 계산
transition_rates = (transition_counts / total_bed_bath_customers) * 100
transition_rates_df = transition_rates.reset_index()
transition_rates_df.columns = ['Next Category', 'Transition Rate (%)']

# 결과 출력
transition_rates_df

# **STEP04**

**고객이 가장 많이 이동하는 카테고리 구하기**

STEP 3과 다르게 장기적인 관점에서 본 전환건수

bed_bath_table을 구매 후 기간을 두지 않고 다른 카테고리를 구매한 건수를 의미함

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Brazilian E-Commerce Public Dataset/Step02.csv')

#원본 백업
df_step02 = df.copy()

In [ ]:
# 특정 카테고리(bed_bath_table) 구매 고객 리스트
bbt_customers = df[df["product_category_name"] == "bed_bath_table"]["customer_unique_id"].unique()

# 해당 고객들의 전체 구매 내역 가져오기
bbt_followup_purchases = df[df["customer_unique_id"].isin(bbt_customers)]

# 'bed_bath_table'이 아닌 다른 카테고리의 구매 건수 집계
bbt_conversion_counts = (
    bbt_followup_purchases[bbt_followup_purchases["product_category_name"] != "bed_bath_table"]
    ["product_category_name_english"].value_counts()
)

# 상위 10개 카테고리 출력
bbt_conversion_counts.head(5)

In [ ]:
# 시각화를 위한 라이브러리 불러오기
import matplotlib.pyplot as plt
import seaborn as sns

# Seaborn 스타일 설정
sns.set_style("whitegrid")

# 그래프 그리기
plt.figure(figsize=(10, 5))
ax = sns.barplot(x=bbt_conversion_counts.head(5).values, y=bbt_conversion_counts.head(5).index, palette="coolwarm")

# 레이블 추가
plt.xlabel("Number of Purchases")
plt.ylabel("Product Category")
plt.title("Top 5 Categories Customers Convert to from Bed_Bath_Table")

# 수치 정확하게 표시
for index, value in enumerate(bbt_conversion_counts.head(5).values):
    ax.text(value + 2, index, f"{value}", va='center', fontsize=12)

plt.xlim(0, max(bbt_conversion_counts.head(10)) + 20)
plt.show()


### 카이제곱검증 수행
특정 카테고리가 단순히 인기 있는 것인지, Bed_Bath_Table 구매자에게 특별한 연관성이 있는지 알기 위함

In [ ]:
from scipy.stats import chi2_contingency

# Bed_Bath_Table 구매자 중 Furniture_Decor 추가 구매 고객 수
bbt_furniture_count = bbt_followup_purchases[bbt_followup_purchases["product_category_name"] == "furniture_decor"]["customer_unique_id"].nunique()

# 비구매자 중 Furniture_Decor 구매 고객 수
non_bbt_furniture_count = df[
    (df["product_category_name"] == "furniture_decor") &
    (~df["customer_unique_id"].isin(bbt_customers))
]["customer_unique_id"].nunique()

# 전체 Bed_Bath_Table 구매 고객 수
bbt_total = len(bbt_customers)

# 전체 고객 수 중 Bed_Bath_Table 미구매자 수
non_bbt_total = len(df["customer_unique_id"].unique()) - bbt_total

# 교차표 구성
contingency_table = [
    [bbt_furniture_count, bbt_total - bbt_furniture_count],  # Bed_Bath_Table 구매자
    [non_bbt_furniture_count, non_bbt_total - non_bbt_furniture_count]  # 비구매자
]

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(contingency_table)

# 결과 출력
print(f"카이제곱 통계량: {chi2:.4f}, p-value: {p:.10f}")

if p < 0.05:
    print("→ Bed_Bath_Table 구매자와 Furniture_Decor 구매 간 유의미한 관계가 있음.")
else:
    print("→ Bed_Bath_Table 구매자와 Furniture_Decor 구매 간 유의미한 관계가 없음.")


### 전환이 유의미한지 확인하기 위해 전환율 확인

In [ ]:
# Bed_Bath_Table을 구매한 전체 고객 수
bbt_total_customers = len(bbt_customers)

# 각 카테고리별 전환율 계산 (Bed_Bath_Table 구매자 중 특정 카테고리도 구매한 고객 비율)
bbt_conversion_rates = (
    bbt_followup_purchases[bbt_followup_purchases["product_category_name"] != "bed_bath_table"]
    .groupby("product_category_name")["customer_unique_id"].nunique() / bbt_total_customers
) * 100

# 전환율 상위 5개 카테고리 확인
bbt_conversion_rates = bbt_conversion_rates.sort_values(ascending=False)

# 결과 출력
bbt_conversion_rates.head(5)


Bed_Bath_Table 고객이 가장 많이 이동하는 카테고리가 furniture_decor인 것은 맞지만, 1.7%에 불과해 전환율을 높일 방안 강구가 남은 숙제.